<a href="https://colab.research.google.com/github/rukmals/ColabProject/blob/master/extratreeclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [4]:
malware = pd.read_csv('permission.csv')

data_in = malware.drop(['android','type'],axis=1).values
labels = malware['type'].values
extratrees = ExtraTreesClassifier().fit(data_in,labels)
select = SelectFromModel(extratrees,prefit=True)
data_in_new = select.transform(data_in)
print(data_in.shape,data_in_new.shape,malware['type'].values.shape)

(398, 329) (398, 38) (398,)


In [15]:
import numpy as np
features = data_in_new.shape[1]
importance = extratrees.feature_importances_
indices = np.argsort(importance)[::-1]

for f in range(features):
    print("%d"%(f+1),malware.columns[2+indices[f]],importance[indices[f]])

1 android.permission.READ_PRECISE_PHONE_STATE 0.18632463126468304
2 android.permission.READ_SOCIAL_STREAM 0.0732861415169372
3 android.permission.WRITE_SOCIAL_STREAM 0.06760923287175442
4 android.permission.ACCESS_NOTIFICATIONS 0.05932343948675095
5 android.permission.ACCESS_WIMAX_STATE 0.055944468183242474
6 android.permission.CHANGE_WIMAX_STATE 0.05402629964015715
7 android.permission.INVOKE_CARRIER_SETUP 0.04599466059463018
8 android.permission.RECEIVE_DATA_ACTIVITY_CHANGE 0.0330447544283561
9 android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY 0.028813016314629944
10 android.permission.SERIAL_PORT 0.027465735939513423
11 android.permission.INTERACT_ACROSS_USERS 0.027127786342600033
12 android.permission.WRITE_GSERVICES 0.025039746370528183
13 android.permission.RECEIVE_WAP_PUSH 0.02164432496257588
14 android.permission.READ_DREAM_STATE 0.0196133739288375
15 android.permission.WRITE_DREAM_STATE 0.018191657717215457
16 android.permission.ACCESS_FM_RADIO 0.016610015474672897
17 and

In [16]:
from sklearn.ensemble import RandomForestClassifier
legit_train,legit_test,mal_train,mal_test = train_test_split(data_in_new,labels,test_size=0.2)
classif = RandomForestClassifier(n_estimators=50)
classif.fit(legit_train,mal_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import confusion_matrix
result = classif.predict(legit_test)
conf_nat = confusion_matrix(mal_test,result)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

grad_boost = GradientBoostingClassifier(n_estimators = 50)

grad_boost.fit(legit_train,mal_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [14]:
grad_boost.score(legit_test,mal_test)*100

95.0